In [8]:
import csv
import re
import pandas as pd
import numpy as np
def load_embeddings():
    """
    Loads pretrained embeddings from a file and returns
    the list of words, a numpy matrix with each row
    containing the respective embedding of the word, and a 
    dictionary with key:value as word:embedding.
    """
    f = open("glove.6B.50d.txt",'r')
    vocab = {}
    words = []
    vectors = []
    for line in f:
        tokens = line.split()
        word = tokens[0]
        embedding = np.array([float(val) for val in tokens[1:]])
        words.append(word)
        vectors.append(embedding)
        vocab[word] = embedding
    return words,np.asarray(vectors),vocab

WORDS, VECTORS, VOCAB = load_embeddings()

train_df = pd.read_csv("mini.tsv", sep="\t",error_bad_lines=False) #skipped 218 lines
# print(train_df)

In [9]:
VOCAB["unk"]

array([-7.9149e-01,  8.6617e-01,  1.1998e-01,  9.2287e-04,  2.7760e-01,
       -4.9185e-01,  5.0195e-01,  6.0792e-04, -2.5845e-01,  1.7865e-01,
        2.5350e-01,  7.6572e-01,  5.0664e-01,  4.0250e-01, -2.1388e-03,
       -2.8397e-01, -5.0324e-01,  3.0449e-01,  5.1779e-01,  1.5090e-02,
       -3.5031e-01, -1.1278e+00,  3.3253e-01, -3.5250e-01,  4.1326e-02,
        1.0863e+00,  3.3910e-02,  3.3564e-01,  4.9745e-01, -7.0131e-02,
       -1.2192e+00, -4.8512e-01, -3.8512e-02, -1.3554e-01, -1.6380e-01,
        5.2321e-01, -3.1318e-01, -1.6550e-01,  1.1909e-01, -1.5115e-01,
       -1.5621e-01, -6.2655e-01, -6.2336e-01, -4.2150e-01,  4.1873e-01,
       -9.2472e-01,  1.1049e+00, -2.9996e-01, -6.3003e-03,  3.9540e-01])

In [12]:
max_len = 0
max_index=0
for index, row in train_df.iterrows():
    if isinstance(row["Text"], str) and len(re.findall(r'^-+|\w+|\S+', row["Text"]))>max_len:
        # print(row["Text"].split())
        # print(re.findall(r'^-+|\w+|\S+', row["Text"]))
        # print(re.findall(r"[\w']+|[.,!?;]", row["Text"]))
        max_len = len(re.findall(r'^-+|\w+|\S+', row["Text"]))
        max_index = index
print("max_len",max_len)
print("max_index",max_index)

max_len 71
max_index 1062


In [13]:
prev_row=None
first = True
encoder_input_data_empty = True
decoder_input_data_empty = True
encoder_input_data = None
decoder_input_data = None
for index, row in train_df.iterrows():
    if not first and isinstance(prev_row["Text"], str) and isinstance(row["Text"], str) and int(prev_row["LineID"][1:]) == int(row["LineID"][1:]) + 1:
        vec = np.zeros((max_len,50))
        words = re.findall(r'^-+|\w+|\S+', prev_row["Text"])
        for i in range(len(words)):
            if words[i] in VOCAB:
                vec[i] = VOCAB[words[i]]
            else:
                vec[i] = VOCAB['unk']
        if encoder_input_data_empty:
            encoder_input_data = vec
            encoder_input_data_empty = False
        else:
            encoder_input_data = np.vstack((encoder_input_data,vec))
        vec = np.zeros((max_len,50))
        words = re.findall(r'^-+|\w+|\S+', row["Text"])
        for i in range(len(words)):
            if words[i] in VOCAB:
                vec[i] = VOCAB[words[i]]
            else:
                vec[i] = VOCAB['unk']
        if decoder_input_data_empty:
            decoder_input_data = vec
            decoder_input_data_empty = False
        else:
            decoder_input_data = np.vstack((decoder_input_data,vec))
    prev_row = row
    first = False
print(encoder_input_data.shape)
print(decoder_input_data.shape)
    

(59214, 50)
(59214, 50)


False
